In [ ]:
import huracanpy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Polygon, MultiPolygon, Point
import geopandas as gpd
from glob import glob

In [ ]:
ls ../TC_tracks/

In [ ]:
# - LOAD TRACKS
tracks={}

# UM
um_files = glob("../TC_tracks/um_*")
for f in um_files:
    name = f.split("/")[-1].split(".")[0]
    tracks[name] = huracanpy.load(f)
    tracks[name] = tracks[name].rename({"sfcwind_max":"wind", "psl_min":"slp"})

# IFS files
ifs_files = glob("../TC_tracks/ifs_*")
for f in ifs_files:
    name = f.split("/")[-1].split(".")[0]
    tracks[name] = huracanpy.load(f).rename({"psl":"slp"})

## Plot most intense track

In [ ]:
# TODO: Expand the box and center on storm
# TODO: Make annotations smarter
# TODO: Make it possible to add places

def plot_track(track, name, season, 
               xytext = (-15,15), buffer = 2, freq_annot = 1):
    
    cmap = "Spectral_r" #cm.buda

    # Coordinates from track
    lon = np.where(track.lon < 0, track.lon + 360, track.lon)
    lon_min, lon_max = lon.min(), lon.max()
    lat_min, lat_max = track.lat.min().values, track.lat.max().values
    lon_mid, lat_mid = (lon_min+lon_max)/2, (lat_min+lat_max)/2
    
    # Map setup
    ## Matplotlib
    fig = plt.figure(figsize = [8,8])
    ax = plt.axes(projection = ccrs.NearsidePerspective(central_longitude=lon_mid, central_latitude=lat_mid,))
    plt.style.use('dark_background')
    plt.rcParams["font.family"] = "monospace"
    ax.set_title(name, fontweight = "bold", loc = "left", family = "monospace")
    ## Cartopy
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.OCEAN, color = "w", alpha = 0.5)
    ax.add_feature(cfeature.LAND, color = "w", alpha = 0.9)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, color = 'w', alpha = 0.6,)
    gl.top_labels = False
    gl.right_labels = False
    
    # Plot track
    ## Line and colorbar
    l = huracanpy.plot.fancyline(track.lon, track.lat, track.wind, cmap = cmap)
    cbar = plt.colorbar(l, shrink = 0.33, ax = ax, location = "right",)
    cbar.set_label(label='Maximum wind speed / m/s',family = "monospace")
    for t in np.array([35, 64, 83, 96, 113, 137])/1.95: # SSHS
        cbar.ax.plot([0,1], [t, t], color = 'k') # Add marker for SSHS thresholds
    ## Scatter and legend
    s = sns.scatterplot(data = track, x = "lon", y = "lat", hue = "wind", palette = cmap,
                    transform = ccrs.PlateCarree(), zorder = 10, legend = False, edgecolor = 'k',)
    #h, l = s.get_legend_handles_labels()
    #for text in l.get_texts(): text.set_color('k'); text.set_family("monospace")
    ax.set_extent([lon_min - buffer/2, lon_max + buffer/2, lat_min -2 - buffer*2, lat_max +buffer*2])
    plt.tight_layout()

    plt.savefig("fancy_figs/most_intense_TC_"+name+".png")
    return ax

In [ ]:
for n in tracks:
    t=tracks[n]
    most_intense_id=t.sortby("wind").isel(record = -1).track_id
    most_intense_track = t.hrcn.sel_id(most_intense_id)
    plot_track(most_intense_track, n, 2020)

## Plot most intense track's structure

In [ ]:
n="um_glm_n1280_GAL9"
tracks[n]["record"] = tracks[n].record
t=tracks[n]
most_intense_id=t.sortby("wind").isel(record = -1).track_id
most_intense_track = t.hrcn.sel_id(most_intense_id)

In [ ]:
most_intense_track.record

In [ ]:
# Precip
varname = "pr"
var = xr.open_dataset("../snapshots/"+varname+"/"+n+"_"+varname+".nc", decode_times = False)["snap_"+varname]
var.sel(snapshot = most_intense_track.record).plot(col = "record", col_wrap = 5, cmap = "YlGnBu", vmax = 12e-3)

In [ ]:
# wind
varname = "uas"
uas = xr.open_dataset("../snapshots/"+varname+"/"+n+"_"+varname+".nc", decode_times = False)["snap_"+varname]
varname = "vas"
vas = xr.open_dataset("../snapshots/"+varname+"/"+n+"_"+varname+".nc", decode_times = False)["snap_"+varname]
#var.sel(snapshot = most_intense_track.record).plot(col = "record", col_wrap = 5, cmap = "YlGnBu", vmax = 12e-3)

In [ ]:
ws = np.sqrt(uas**2 + vas**2)
ws.sel(snapshot = most_intense_track.record[20:]).plot(col = "record", col_wrap = 5, cmap = "Spectral_r", vmax = 50)

In [ ]:
snaps = ws.sel(snapshot = most_intense_track.record[20:])
for i in range(len(snaps.record)):
    plt.figure()
    snaps.isel(record = i).plot(vmin = 0, vmax = 50, cmap = "Spectral_r")
    plt.title("Maximum wind speed in m/s")
    plt.savefig("snapshots/wind_speed/"+n+"_"+str(i)+".png")